In [1]:
!pip install techieray-ai-reg-tracker-api


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [2]:
from techieray_ai_reg_tracker_api import GlobalAIRegulationTrackerClient

api_key = "APIキー"
client = GlobalAIRegulationTrackerClient(api_key)


NOTE: The following terms apply to your access and use of the API service to the Global AI Regulation Tracker, which was developed by Raymond Sun (techie_ray) (Developer)), during the free trial period (as indicated within the relevant API key provided by Developer). Developer retains all intellectual property rights in the API service, including the implementation code, the API key, and the outputs returned by the API (subject to any third party rights). Outputs returned by the API may contain links to third party sources (i.e. the "href" attribute in the output). Your access to those third party sources are at your own risk. The Developer does not claim copyright to those third party sources, but owns the copyright in the summary relating to the third party source (i.e. the "desc" attribute). You may use the API service (and its outputs), free of charge, solely for personal use or internal business use throughout the free trial period. At this stage, the API service is not designed (

In [34]:
# API確認
response = client.call(market="US", target_news="acts_bills_reform", target_date="")
print(type(response))
print(response[:2])  

<class 'list'>
[{'desc': '<News summary only available for Tier 2 subscription plan or above>', 'href': 'https://www.congress.gov/bill/118th-congress/house-bill/7832/text', 'label': 'Emerging Innovative Border Technologies Act:'}, {'desc': '<News summary only available for Tier 2 subscription plan or above>', 'href': 'https://www.congress.gov/bill/118th-congress/house-bill/5077?q=%7B%22search%22%3A%22Artificial+Intelligence%22%7D&s=2&r=2', 'label': 'Creating Resources for Every American To Experiment with Artificial Intelligence Act of 2023:'}]


In [33]:
# APIのofficial_materialsの中身を確認
response = client.call(market="GB", target_news="official_materials", target_date="")
for i, item in enumerate(response if isinstance(response, list) else response.get("newsItems", []) if isinstance(response, dict) else []):
    if isinstance(item, dict):
        print(f"--- No.{i+1} フィールド一覧 ---")
        print(item.keys())

--- No.1 フィールド一覧 ---
dict_keys(['desc', 'href', 'label'])
--- No.2 フィールド一覧 ---
dict_keys(['desc', 'href', 'label'])
--- No.3 フィールド一覧 ---
dict_keys(['desc', 'href', 'label'])


In [35]:
from techieray_ai_reg_tracker_api import GlobalAIRegulationTrackerClient
import pandas as pd

In [36]:
# 国リスト（リストAde）
target_countries = {
    "EU": "European Union",
    "US": "United States",
    "GB": "United Kingdom",
    "JP": "Japan",
    "CN": "China",
    "KR": "South Korea",
    "IN": "India",
}

# APIカテゴリと英語ラベルの対応
category_labels = {
    "acts_bills_reform": "Law",
    "guidelines_standards_frameworks": "Guideline",
    "orders_admin_regs": "Order",
    "official_materials": "Strategy"
}

# DataFrameカラム
columns = [
    "regulation_name",       # 政策名
    "country",               # 国名
    "announced_date",        # 発表日・成立日
    "enforcement_date",      # 施行日
    "category",              # Law / Guideline / Order / Strategy
    "api_category",          # APIカテゴリ原語
    "reference_point",       # 要約
    "purpose",               # 規制／推進　など（自動判別なら工夫可能）
    "keywords",              # キーワード抜き出し（とりあえず空欄推奨）
    "official_link"          # 公式リンク
]

records = []
for code, name in target_countries.items():
    for api_cat, eng_label in category_labels.items():
        response = client.call(market=code, target_news=api_cat, target_date="")
        # 型対応（list型・dict型どちらでもOK）
        items = (
            response
            if isinstance(response, list)
            else response.get("newsItems", [])
            if isinstance(response, dict)
            else []
        )
        for item in items:
            if isinstance(item, dict):
                records.append({
                    "regulation_name": item.get("label", "").replace(":", "").strip(),
                    "country": name,
                    "announced_date": item.get("announcedDate", ""),
                    "enforcement_date": item.get("enforcementDate", ""),
                    "category": eng_label,
                    "api_category": api_cat,
                    "reference_point": item.get("desc", ""),
                    "purpose": "",      # 目的・性格。手動補記orNLP自動判定も可
                    "keywords": "",     # 関連ワード。必要あれば後処理で
                    "official_link": item.get("href", ""),
                })

# DataFrame化
df = pd.DataFrame(records, columns=columns)
print(df.head())

                                     regulation_name        country  \
0        Emerging Innovative Border Technologies Act  United States   
1  Creating Resources for Every American To Exper...  United States   
2  Small Business Artificial Intelligence Trainin...  United States   
3  AI Leadership To Enable Accountable Deployment...  United States   
4  Small Business Artificial Intelligence Advance...  United States   

  announced_date enforcement_date category       api_category  \
0                                      Law  acts_bills_reform   
1                                      Law  acts_bills_reform   
2                                      Law  acts_bills_reform   
3                                      Law  acts_bills_reform   
4                                      Law  acts_bills_reform   

                                     reference_point purpose keywords  \
0  <News summary only available for Tier 2 subscr...                    
1  <News summary only available for 

In [37]:
df.to_csv("ai_regulation_list.csv", index=False)

In [14]:
# 国リスト（リストAde）
target_countries = {
    "Europe": "European Union",
}

# APIカテゴリと英語ラベルの対応
category_labels = {
    "acts_bills_reform": "Law",
    "guidelines_standards_frameworks": "Guideline",
    "orders_admin_regs": "Order",
    "official_materials": "Strategy"
}

# DataFrameカラム
columns = [
    "regulation_name",       # 政策名
    "country",               # 国名
    "announced_date",        # 発表日・成立日
    "enforcement_date",      # 施行日
    "category",              # Law / Guideline / Order / Strategy
    "api_category",          # APIカテゴリ原語
    "reference_point",       # 要約
    "purpose",               # 規制／推進　など（自動判別なら工夫可能）
    "keywords",              # キーワード抜き出し（とりあえず空欄推奨）
    "official_link"          # 公式リンク
]

records = []
for code, name in target_countries.items():
    for api_cat, eng_label in category_labels.items():
        response = client.call(market=code, target_news=api_cat, target_date="")
        # 型対応（list型・dict型どちらでもOK）
        items = (
            response
            if isinstance(response, list)
            else response.get("newsItems", [])
            if isinstance(response, dict)
            else []
        )
        for item in items:
            if isinstance(item, dict):
                records.append({
                    "regulation_name": item.get("label", "").replace(":", "").strip(),
                    "country": name,
                    "announced_date": item.get("announcedDate", ""),
                    "enforcement_date": item.get("enforcementDate", ""),
                    "category": eng_label,
                    "api_category": api_cat,
                    "reference_point": item.get("desc", ""),
                    "purpose": "",      # 目的・性格。手動補記orNLP自動判定も可
                    "keywords": "",     # 関連ワード。必要あれば後処理で
                    "official_link": item.get("href", ""),
                })

# DataFrame化
df = pd.DataFrame(records, columns=columns)
print(df.head())

                                   regulation_name         country  \
0        [WITHDRAWN] AI Liability Directive (2022)  European Union   
1  New Product Liability Directive (2022/0302/COD)  European Union   
2                     EU AI Act (2024/1689) (2024)  European Union   
3       General-Purpose AI Code of Practice (2025)  European Union   
4     Guidelines on prohibited AI practices (2025)  European Union   

  announced_date enforcement_date   category                     api_category  \
0                                        Law                acts_bills_reform   
1                                        Law                acts_bills_reform   
2                                        Law                acts_bills_reform   
3                                  Guideline  guidelines_standards_frameworks   
4                                  Guideline  guidelines_standards_frameworks   

                                     reference_point purpose keywords  \
0  <News summary on

In [15]:
df.to_csv("ai_regulation_listEU.csv", index=False)

In [9]:
# 一度だけ確認
response = client.call(market="FR", target_news="acts_bills_reform", target_date="")
print("\n=== Raw response (France - Law) ===")
print(type(response))
print(response)



=== Raw response (France - Law) ===
<class 'list'>
['Error - the key "acts_bills_reform" is not currently available for France. Please try another key.']
